# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [2]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score

df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Все столбцы в датафрейме имеют числовые значения, что хорошо подходит для создания и обучения ML-моделей классификации. Для задачи классификации возьмем логистическую регрессию и дерево решений. Для поиска наилучших гиперпараметров также импортируем GridSearchCV.

## Разбейте данные на выборки

In [4]:
#отбираем фичи
features = df.drop(columns=['is_ultra'])
#забираем целевую переменную для классификации
target = df.is_ultra
#делим датасет на тестовую и обучающую выборку
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=123)
#выделяем из обучающей выборки датасет для валидации
features_train, features_valid, target_train, target_valid = train_test_split(features_train, target_train, test_size=0.25, random_state=123)
print(features.shape, features_train.shape, features_valid.shape, features_test.shape)

(3214, 4) (1928, 4) (643, 4) (643, 4)


В результате разбивки данных на выборки около 2 тыс. строк в тренировочном датасете, и около 650 - в валидационном и тестовом.

## Исследуйте модели

In [5]:
#задаем гиперпараметры для дерева решений
params_clf = {'criterion': ['gini', 'entropy'], 'max_depth': range(1,5)}
#задаем гиперпараметры для логистической регрессии
params_logreg = {'penalty' : ['l2', 'none'],
    'C': np.logspace(-5, 5, 20)}
#задаем гиперпараметры для случайного леса
params_rf_clf = {'n_estimators': range(1,5), 'criterion': ['gini', 'entropy']}

#перебираем модели с помощью GridSearch и обучаем их на тренировочном датасете
logreg = GridSearchCV(LogisticRegression(random_state=123), params_logreg)
best_logreg = logreg.fit(features_train, target_train)

clf = GridSearchCV(DecisionTreeClassifier(random_state=123), params_clf)
best_clf = clf.fit(features_train, target_train)

rf_clf = GridSearchCV(RandomForestClassifier(random_state=123), params_rf_clf)
best_rf_clf = rf_clf.fit(features_train, target_train)

print(best_logreg.best_estimator_, best_clf.best_estimator_, best_rf_clf.best_estimator_)

/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:1322: UserWarning: Setting penalty='none' will ignore the C and l1_ratio parameters
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status

LogisticRegression(C=0.004281332398719391, random_state=123) DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=123) RandomForestClassifier(n_estimators=4, random_state=123)


In [6]:
pred_logreg = best_logreg.predict(features_valid)
pred_clf = best_clf.predict(features_valid)
pred_rf_clf = best_rf_clf.predict(features_valid)
print(f'Accuracy LogisticRegression (validation): {accuracy_score(target_valid, pred_logreg)}')
print(f'Accuracy DTClassifier (validation): {accuracy_score(target_valid, pred_clf)}')
print(f'Accuracy RFClassifier (validation): {accuracy_score(target_valid, pred_rf_clf)}')

Accuracy LogisticRegression (validation): 0.7340590979782271
Accuracy DTClassifier (validation): 0.7838258164852255
Accuracy RFClassifier (validation): 0.7900466562986003


**Что сделано:**
- у дерева решений протестированы гиперпараметры критерия ветвления, а также максимальная глубина дерева
- у логистической регрессии протестированы гиперпараметры регуляризации (C) и вида регуляризация (penalty), используемые для штрафования алгоритма за неверно классифицированный объект

**Полученные лучшие модели логистической регрессии и дерева решений:**
- LogisticRegression(C=0.004281332398719391, random_state=123) 
- DecisionTreeClassifier(criterion='entropy', max_depth=3, random_state=123)
- RandomForestClassifier(n_estimators=4, random_state=123)

**Точность лучших моделей на валидационной выборке:**
- логистическая регрессия - 0.7340590979782271
- дерево решений - 0.7838258164852255
- случайный лес - 0.7900466562986003

## Проверьте модель на тестовой выборке

In [7]:
pred_logreg = best_logreg.predict(features_test)
pred_clf = best_clf.predict(features_test)
pred_rf_clf = best_rf_clf.predict(features_test)
print(f'Accuracy LogisticRegression (test): {accuracy_score(target_test, pred_logreg)}')
print(f'Accuracy DTClassifier (test): {accuracy_score(target_test, pred_clf)}')
print(f'Accuracy RFClassifier (test): {accuracy_score(target_test, pred_rf_clf)}')

Accuracy LogisticRegression (test): 0.7511664074650077
Accuracy DTClassifier (test): 0.8055987558320373
Accuracy RFClassifier (test): 0.7791601866251944


На тестовой выборке дерево решений и логистическая регрессия показали себя лучше, чем на валидационной. Точность случайного леса снизилась. Наибольшая точность у дерева решений.

## (бонус) Проверьте модели на адекватность

In [9]:
#делаем случайную модель
np.random.seed(123)
random_pred = np.random.randint(2, size=len(target_test))
print(f'Точность случайной классификации: {accuracy_score(target_test, random_pred)}')

Точность случайной классификации: 0.49611197511664074


Случайная модель показала себя хуже, в связи с чем можно сделать вывод об адекватности моделей с подобранными гиперпараметрами.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
